In [1]:
from sqlalchemy.ext.declarative import declarative_base

In [2]:
base = declarative_base() 

In [3]:
from sqlalchemy import Column, Integer, String, ForeignKey

In [4]:
from sqlalchemy.orm import relationship

In [5]:
class Artist(base):
    __tablename__ = "artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    album =relationship("Album", back_populates= "artist")
    
    def __repr__(self):
        return "<T artist(name ='%s')>" % (self.name)

class Album(base):
    __tablename__ = "album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("artist.id"))
    
    artist = relationship("Artist", back_populates = "album", uselist=False)
    
    # 다시 클래스 정의를 해야한다.
    # 1:N 의 관계 리스트 타입이 아니기 때문에 # 꼭 1이여야 한다.  
    # Artist 객체와 관계를 맺고 얘를 추적해서 가면 album(어트리뷰트랑 관련이 있다)을 만날 수 있다 
    
class Genre(base):
    __tablename__ = "genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Track(base):
    __tablename__ = "track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    album_id = Column(Integer, ForeignKey('album.id'))
    genre_id = Column(Integer, ForeignKey('genre.id'))

In [6]:
from sqlalchemy import create_engine

In [7]:
engine = create_engine("sqlite:///test2.db", echo = True)

In [8]:
con = engine.connect()

2018-08-18 17:00:19,549 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-18 17:00:19,552 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 17:00:19,554 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-18 17:00:19,555 INFO sqlalchemy.engine.base.Engine ()


In [9]:
base.metadata.create_all(engine)

2018-08-18 17:00:25,691 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-08-18 17:00:25,692 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 17:00:25,695 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-08-18 17:00:25,696 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 17:00:25,697 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-08-18 17:00:25,698 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 17:00:25,700 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-08-18 17:00:25,701 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 17:00:25,703 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE artist (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2018-08-18 17:00:25,704 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 17:00:25,714 INFO sqlalchemy.engine.base.Engine COMMIT
2018-08-18 17:00:25,715 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE genre (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id

In [10]:
artist1 = Artist(name="Led zepplin")
artist2 = Artist(name="AC/DC")

In [11]:
artist1

<T artist(name ='Led zepplin')>

In [12]:
artist1.album = [Album(title= "IV"), Album(title="who Made Who")]

In [13]:
type(artist1.album[1])

__main__.Album

In [14]:
artist1.album[1].title

'who Made Who'

In [15]:
from sqlalchemy.orm import sessionmaker

In [16]:
Session = sessionmaker(bind = engine)

In [17]:
session = Session()

In [18]:
session.add_all([artist1, artist2])

In [19]:
session.dirty

IdentitySet([])

In [20]:
session.new

IdentitySet([<T artist(name ='Led zepplin')>, <__main__.Album object at 0x000001CCFEE1AE10>, <__main__.Album object at 0x000001CCFEE1A8D0>, <T artist(name ='AC/DC')>])

In [21]:
session.commit()

2018-08-18 17:00:53,536 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 17:00:53,538 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-08-18 17:00:53,539 INFO sqlalchemy.engine.base.Engine ('Led zepplin',)
2018-08-18 17:00:53,543 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-08-18 17:00:53,544 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2018-08-18 17:00:53,546 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-08-18 17:00:53,548 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2018-08-18 17:00:53,549 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-08-18 17:00:53,550 INFO sqlalchemy.engine.base.Engine ('who Made Who', 1)
2018-08-18 17:00:53,552 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
result = con.execute("select * from Genre")

2018-08-18 17:00:57,985 INFO sqlalchemy.engine.base.Engine select * from Genre
2018-08-18 17:00:57,986 INFO sqlalchemy.engine.base.Engine ()


In [23]:
result.fetchall()

[]

In [24]:
artist1.album[0].title 

2018-08-18 17:01:06,327 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 17:01:06,328 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-08-18 17:01:06,329 INFO sqlalchemy.engine.base.Engine (1,)
2018-08-18 17:01:06,332 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-08-18 17:01:06,333 INFO sqlalchemy.engine.base.Engine (1,)


'IV'

In [25]:
artist1

<T artist(name ='Led zepplin')>

In [26]:
session.dirty

IdentitySet([])

In [27]:
session.commit() # 커밋을 반드시 해줘야 반영이 된다 .

2018-08-18 17:01:15,198 INFO sqlalchemy.engine.base.Engine COMMIT


In [28]:
session.add_all([    
    Album(title="1", artist_id=1),
    Album(title="2", artist_id=2)
]) # 메모리상에 올라온 것뿐

In [29]:
session.dirty 

IdentitySet([])

In [30]:
session.add_all([
    Genre(name="Rock"),
    Genre(name="Metal")
]) 

In [31]:
session.add_all([    
    Track(title="1-1", album_id=1, genre_id=1),
    Track(title="1-2", album_id=1, genre_id=2),
    Track(title="2-1", album_id=2, genre_id=1),
    Track(title="2-2", album_id=2, genre_id=2),
])

In [32]:
session.new

IdentitySet([<__main__.Album object at 0x000001CCFEFBEA58>, <__main__.Album object at 0x000001CCFEFBE9E8>, <__main__.Genre object at 0x000001CCFEFBEBE0>, <__main__.Genre object at 0x000001CCFEFBEC50>, <__main__.Track object at 0x000001CCFEFB9390>, <__main__.Track object at 0x000001CCFEFB9400>, <__main__.Track object at 0x000001CCFEFB9470>, <__main__.Track object at 0x000001CCFEFB94E0>])

In [33]:
session.commit()

2018-08-18 17:01:28,041 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 17:01:28,043 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-08-18 17:01:28,044 INFO sqlalchemy.engine.base.Engine ('Rock',)
2018-08-18 17:01:28,049 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-08-18 17:01:28,051 INFO sqlalchemy.engine.base.Engine ('Metal',)
2018-08-18 17:01:28,052 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-08-18 17:01:28,053 INFO sqlalchemy.engine.base.Engine ('1', 1)
2018-08-18 17:01:28,055 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-08-18 17:01:28,055 INFO sqlalchemy.engine.base.Engine ('2', 2)
2018-08-18 17:01:28,058 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-08-18 17:01:28,059 INFO sqlalchemy.engine.base.Engine ('1-1', 1, 1)
2018-08-18 17:01:28,060 INFO sqlalchemy.engi

In [34]:
temp = Track(title="3-1", album_id=3, genre_id=3)

In [35]:
session.add(temp)

In [36]:
session.new

IdentitySet([<__main__.Track object at 0x000001CCFEFBEE80>])

In [37]:
session.commit()

2018-08-18 17:01:40,199 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 17:01:40,201 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-08-18 17:01:40,202 INFO sqlalchemy.engine.base.Engine ('3-1', 3, 3)
2018-08-18 17:01:40,206 INFO sqlalchemy.engine.base.Engine COMMIT


In [38]:
print(temp.id)

2018-08-18 17:01:43,935 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 17:01:43,937 INFO sqlalchemy.engine.base.Engine SELECT track.id AS track_id, track.title AS track_title, track.album_id AS track_album_id, track.genre_id AS track_genre_id 
FROM track 
WHERE track.id = ?
2018-08-18 17:01:43,938 INFO sqlalchemy.engine.base.Engine (5,)
5


In [39]:
result = con.execute("select * from Album")

2018-08-18 17:02:30,599 INFO sqlalchemy.engine.base.Engine select * from Album
2018-08-18 17:02:30,600 INFO sqlalchemy.engine.base.Engine ()


In [41]:
for row in result:
    print(row)

(1, 'IV', 1)
(2, 'who Made Who', 1)
(3, '1', 1)
(4, '2', 2)


In [49]:
result = session.query(Track.title)

2018-08-18 17:06:00,078 INFO sqlalchemy.engine.base.Engine SELECT track.title AS track_title 
FROM track
2018-08-18 17:06:00,079 INFO sqlalchemy.engine.base.Engine ()
('1-1',)
('1-2',)
('2-1',)
('2-2',)
('3-1',)
('4-1',)


In [46]:
session.add(Track(title="4-1", album_id=4, genre_id=4))

In [51]:
result = con.execute("select * from Track")

2018-08-18 17:06:14,421 INFO sqlalchemy.engine.base.Engine select * from Track
2018-08-18 17:06:14,422 INFO sqlalchemy.engine.base.Engine ()


In [52]:
for row in result:
    print(row)

(1, '1-1', 1, 1)
(2, '1-2', 1, 2)
(3, '2-1', 2, 1)
(4, '2-2', 2, 2)
(5, '3-1', 3, 3)


In [78]:
from sqlalchemy import join

In [79]:
result = session.query(Track.title, Album.title, Genre.name, Artist.name).select_from(Track).join(Album).join(Genre).join(Artist).all()

2018-08-09 02:36:15,913 INFO sqlalchemy.engine.base.Engine SELECT track.title AS track_title, album.title AS album_title, genre.name AS genre_name, artist.name AS artist_name 
FROM track JOIN album ON album.id = track.album_id JOIN genre ON genre.id = track.genre_id JOIN artist ON artist.id = album.artist_id
2018-08-09 02:36:15,915 INFO sqlalchemy.engine.base.Engine ()


In [80]:
for row in result:
    print(row)

('1-1', 'IV', 'Rock', 'Led zepplin')
('1-2', 'IV', 'Metal', 'Led zepplin')
('2-1', 'who Made Who', 'Rock', 'Led zepplin')
('2-2', 'who Made Who', 'Metal', 'Led zepplin')
('3-1', '1', 'Rock', 'Led zepplin')
('1-1', 'IV', 'Rock', 'Led zepplin')
('1-2', 'IV', 'Metal', 'Led zepplin')
('2-1', 'who Made Who', 'Rock', 'Led zepplin')
('2-2', 'who Made Who', 'Metal', 'Led zepplin')
('3-1', '1', 'Rock', 'Led zepplin')
('1-1', 'IV', 'Rock', 'Led zepplin')
('1-2', 'IV', 'Metal', 'Led zepplin')
('2-1', 'who Made Who', 'Rock', 'Led zepplin')
('2-2', 'who Made Who', 'Metal', 'Led zepplin')
('3-1', '1', 'Rock', 'Led zepplin')
('1-1', 'IV', 'Rock', 'Led zepplin')
('1-2', 'IV', 'Metal', 'Led zepplin')
('2-1', 'who Made Who', 'Rock', 'Led zepplin')
('2-2', 'who Made Who', 'Metal', 'Led zepplin')
('3-1', '1', 'Rock', 'Led zepplin')


In [81]:
sonList = [dict(Track=row[0], Album=row[1], Genre=row[2], Artist=row[3]) for row in result]

In [82]:
sonList

[{'Album': 'IV', 'Artist': 'Led zepplin', 'Genre': 'Rock', 'Track': '1-1'},
 {'Album': 'IV', 'Artist': 'Led zepplin', 'Genre': 'Metal', 'Track': '1-2'},
 {'Album': 'who Made Who',
  'Artist': 'Led zepplin',
  'Genre': 'Rock',
  'Track': '2-1'},
 {'Album': 'who Made Who',
  'Artist': 'Led zepplin',
  'Genre': 'Metal',
  'Track': '2-2'},
 {'Album': '1', 'Artist': 'Led zepplin', 'Genre': 'Rock', 'Track': '3-1'},
 {'Album': 'IV', 'Artist': 'Led zepplin', 'Genre': 'Rock', 'Track': '1-1'},
 {'Album': 'IV', 'Artist': 'Led zepplin', 'Genre': 'Metal', 'Track': '1-2'},
 {'Album': 'who Made Who',
  'Artist': 'Led zepplin',
  'Genre': 'Rock',
  'Track': '2-1'},
 {'Album': 'who Made Who',
  'Artist': 'Led zepplin',
  'Genre': 'Metal',
  'Track': '2-2'},
 {'Album': '1', 'Artist': 'Led zepplin', 'Genre': 'Rock', 'Track': '3-1'},
 {'Album': 'IV', 'Artist': 'Led zepplin', 'Genre': 'Rock', 'Track': '1-1'},
 {'Album': 'IV', 'Artist': 'Led zepplin', 'Genre': 'Metal', 'Track': '1-2'},
 {'Album': 'who Made 